<a href="https://colab.research.google.com/github/kaleetock/exoplanets/blob/main/Exoplanet_Watch_Target_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#@title Install modules, mount Gdrive, define functions
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.time import Time
from astropy.io import fits
import json
import math

#######################################################

def make_ra_dict(df):
  ras = {}
  for ind in range(len(df.index)):
    ra, dec = df.iloc[ind]['ra'], df.iloc[ind]['dec']
    planet = df.iloc[ind]['pl_name']
    ras[planet] = str(f"{int(ra)},{int(dec)}")
  return(ras)

#######################################################

def best_ra():
  nt = Time.now()
  m = re.search(r"B\d\d\d\d\.(\d\d\d)", nt.byear_str)
  yr_frac = int(m.group(1))
  if (yr_frac - 726) < 0:   # 726/1000 = yr_frac Sept equinox (best ra = 0 deg)
    yr_frac += 1000

  best_ra_deg = (yr_frac - 726)*.36

  min_ra = best_ra_deg - 95  # best ra is within about 75 degrees of the zenith RA
  if min_ra < 0:
    min_ra += 360

  max_ra = best_ra_deg + 95
  if max_ra > 360:
    max_ra -= 360

  return(round(best_ra_deg,1), round(min_ra, 1), round(max_ra,1))

#########################################################

def get_next_transits (t0, t0_unc, period, period_unc):
  nt = Time.now()

  next_epoch = math.ceil((nt.jd - t0)/period)

  next_transits = []
  for i in range(-2, 30):
    epoch = next_epoch + i
    t = round((t0 + (epoch*period)),5)    # Equation 1 from Zellem et. al., 2020
#    if next_epoch == 1425:
#      print(f"epoch {epoch} t0 {t0} period {period} t {t}")
    time = Time(t, format='jd')
    calc_unc = math.sqrt(((epoch**2) * (period_unc**2)) + (t0_unc**2))   # Assume cross term zero from Zellem et. al.,2020
    calc_unc = round(calc_unc, 6)
    next_transits.append({"jd": t, "date": time.iso, "epoch": epoch, "calc": t, "unc": calc_unc})
  return(next_transits)

#################################################

def upcoming_transits(planet, t0, t0_unc, period, period_unc, ra, dec, num_to_show, window):
  out_str = ""
  nt = Time.now()
#  print ("The time now is %.6f." % nt.jd)

  next_transits = get_next_transits(t0, t0_unc, period, period_unc)
  if (len(next_transits) > 0):
    out_str = (f"Based on {planet} parameters midpoint {round(t0,4)} +/- {round(t0_unc,6)} period {round(period,4)} +/- {round(period_unc,8)}:\n")
    for i in range (0, num_to_show):
      ep = next_transits[i]["epoch"]
      ti = next_transits[i]["jd"]
      midtime = Time(ti, format = 'jd')
      midpoint = midtime.iso
      window_start_jd = Time((ti - window), format='jd')
      window_end_jd = Time((ti + window), format='jd')
      window_start = window_start_jd.iso
      window_end = window_end_jd.iso
      un = next_transits[i]["unc"]

      out_str = out_str + str(f"epoch {ep} : midpoint {round(ti,5)} ({window_start} - {window_end}) UTC (uncertainty {round(un*24,2)} hours)\n") 
  out_str = out_str + str("\n")
  return(out_str)

#################################################

def image_requests(df, index, num_to_show):
  ra, dec = df.iloc[index]['ra'], df.iloc[index]['dec']
  planet = df.iloc[index]['pl_name']
  rank = df.iloc[index]['rank']
  t0 = df.iloc[index]['pl_tranmid']
  t0_unc = df.iloc[index]['pl_tranmiderr1']
  period = df.iloc[index]['pl_orbper']
  period_unc = df.iloc[index]['pl_orbpererr1']
  mag = df.iloc[index]['sy_vmag']
  bmag = df.iloc[index]['sy_bmag']
  transit_depth = df.iloc[index]['pl_trandep']
  transit_duration = df.iloc[index]['pl_trandur']
  spectype = df.iloc[index]['st_spectype']
  plx = df.iloc[index]['sy_plx']

  exp_time = round(60 * (2.512**(mag - 12.5)),0) # seconds
  t_image = exp_time + 14
  baseline = 2                          # hours
  wiggle = 0.5                          # hours
  n_images = 3600*(transit_duration + baseline)/t_image 
  window = (0.5 * transit_duration) + (0.5 * baseline) + (0.5 * wiggle)  # hours
  window = window/24.0   # days

  out = str(f"Rank {rank} planet {planet} ({round(ra,4)}, {round(dec,4)})\nstar mag {round(mag,1)}\n")
  out = out + str(f"parallax {round(plx,2)} Bmag {round(bmag,1)}, Vmag {round(mag,1)}, B-V {round((bmag-mag),1)}, Spectral type {spectype}\n")
  out = out + str(f"t0 {round(t0,4)} +/- {round(t0_unc,6)}\nperiod {round(period,4)} +/- {round(period_unc,6)}\n")
  out = out + str(f"transit depth {round(transit_depth,3)}\ntransit duration {round(transit_duration,2)}\n")
  out = out + str(f"Request {int(n_images)} images with exposure time {int(exp_time)} s.\n")
  out = out + upcoming_transits(planet, t0, t0_unc, period, period_unc, ra, dec, num_to_show, window)
  return(out, planet)

###################################

def show_imageable_planets(planet_dict):
  planet_str, num_planets = "", 0
  for planet in sorted(planet_dict.keys()):
    num_planets += 1
    planet_str = planet_str + str(f"{planet}\t")
    if (len(planet) < 8):
      planet_str = planet_str + "\t"
    if (num_planets%6 == 0):
      planet_str = planet_str + "\n"
  print("Eligible Planets:\n\n"+planet_str)

###################################

def filter_data(df, min_mag, max_mag, max_trandur, max_period, min_transit_depth, num_to_show):
  best_ra_deg, min_ra, max_ra = best_ra()
  print(f"{len(df.index)} rows to start")

  ra_condition = str(f"if abs(dec) < 50deg, ra > {min_ra}deg OR ra < {max_ra}deg")
  ra_condition2 = str(f"if abs(dec) < 50deg, ({min_ra}deg < ra < {max_ra}deg)")
  if (min_ra > max_ra):
    df = df[((df['ra'] > min_ra) | (df['ra'] < max_ra)) | (abs(df['dec']) > 50)]
  else:
    df = df[((df['ra'] > min_ra) & (df['ra'] < max_ra)) | (abs(df['dec']) > 50)]
    ra_condition = ra_condition2

  print(f"{len(df.index)} rows with appropriate RA: {ra_condition}")
  df = df[(df['sy_vmag']) < max_mag]
  df = df[(df['sy_vmag']) > min_mag]
  print(f"{len(df.index)} rows with appropriate RA and mag between {min_mag} and {max_mag}")
  df = df[(df['pl_trandur']) < max_trandur]
  print(f"{len(df.index)} rows with appropriate RA, mag btwn {min_mag} and {max_mag}, and transit duration < {max_trandur} hours")
  df = df[(df['pl_orbper']) < max_period]
  print(f"{len(df.index)} rows with appropriate RA, mag btwn {min_mag} and {max_mag}, transit duration < {max_trandur} hours, and period < {max_period} days")
  df = df[(df['pl_trandep']) > min_transit_depth]
  print(f"{len(df.index)} rows with appropriate RA, mag btwn {min_mag} and {max_mag}, transit duration < {max_trandur} hours, period < {max_period} days, and transit depth > {min_transit_depth}\n")

  planet_dict = {}
  for ind in range(len(df.index)):
    image_request_params, planet = image_requests(df, ind, num_to_show)
    planet_dict[planet] = image_request_params
  show_imageable_planets(planet_dict)
  return(planet_dict)

#############################################################

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [11]:
# Find the path to your data file using the folder icon at left.
# Right-click the file to copy to the path into the string below.

df = pd.read_csv("/content/drive/MyDrive/Exoplanets/Target Selection/Exoplanet Watch Targets.csv")
ras = make_ra_dict(df)
df.head()

,rank,pl_name,pl_letter,hostname,hd_name,hip_name,tic_id,disc_pubdate,disc_year,discoverymethod,disc_locale,disc_facility,disc_instrument,disc_telescope,disc_refname,ra,rastr,dec,decstr,glon,glat,elon,elat,ra_reflink,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbperstr,pl_orbper_reflink,pl_orblpererr1,pl_orblper,pl_orblpererr2,pl_orblperlim,pl_orblperstr,pl_orblper_reflink,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,...,st_densstr,st_dens_reflink,st_rad,st_raderr1,st_raderr2,st_radlim,st_radstr,st_rad_reflink,ttv_flag,ptv_flag,tran_flag,rv_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,pul_flag,sy_snum,sy_pnum,sy_mnum,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,pl_nnotes,sy_pm,sy_pmerr1,sy_pmerr2,sy_pmstr,sy_pm_reflink,sy_pmra,sy_pmraerr1,sy_pmraerr2,sy_pmrastr,x,y,z,htm20
0,1,HIP 41378 e,e,HIP 41378,NaN,HIP 41378,TIC 366443426,2016-08,2016,Transit,Space,K2,Kepler CCD Array,0.95 m Kepler Telescope,<a refstr=VANDERBURG_ET_AL__2016 href=https://...,126.615828,08h26m27.80s,10.080371,+10d04m49.33s,214.52630,25.64577,126.46324,-8.84499,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,131.00,61.00,-36.00,0,"<div><span class=supersubNumber"">131</span><sp...",<a refstr=VANDERBURG_ET_AL__2016 href=https://...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.680&plusmn0.064,<a refstr=BERARDO_ET_AL__2019 href=https://ui....,1.34,0.03,-0.03,0,1.343&plusmn0.032,<a refstr=BERARDO_ET_AL__2019 href=https://ui....,0,0,1,0,0,0,0,0,0,0,1,5,0,1,0,0,0,0,0.0,48.142537,0.104124,-0.104124,48.1425366&plusmn;0.1041239,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,-48.14250,0.104124,-0.104124,-48.142500&plusmn;0.104124,-0.587239,0.790262,0.175029,1036383337
1,2,KIC 5951458 b,b,KIC 5951458,NaN,NaN,TIC 121796770,2015-12,2015,Transit,Space,Kepler,Kepler CCD Array,0.95 m Kepler Telescope,<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,288.991622,19h15m57.99s,41.223068,+41d13m23.05s,72.73419,13.23719,302.08759,62.56270,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,1320.10,12401.80,-152.50,0,"<div><span class=supersubNumber"">1320.10</span...",<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"<div><span class=supersubNumber"">0.2578967</sp...",<a refstr=STASSUN_ET_AL__2019 href=https://ui....,1.52,0.82,-0.82,0,1.52&plusmn0.82,<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,3.0,11.485196,0.041218,-0.041218,11.4851964&plusmn;0.0412180,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,7.38060,0.038297,-0.038297,7.3806000&plusmn;0.0382966,0.244772,-0.711207,0.658992,-1395191073
2,3,KIC 3558849 b,b,KIC 3558849,NaN,NaN,TIC 138888805,2015-12,2015,Transit,Space,Kepler,Kepler CCD Array,0.95 m Kepler Telescope,<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,294.949838,19h39m47.96s,38.605187,+38d36m18.67s,72.43389,7.98057,309.36051,58.68276,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,1322.30,386.10,-11.20,0,"<div><span class=supersubNumber"">1322.3</span>...",<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"<div><span class=supersubNumber"">0.48871</span...",<a refstr=Q1_Q17_DR25_KOI_TABLE href=https://e...,1.01,0.11,-0.11,0,1.01&plusmn0.11,<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1.0,3.373926,0.035656,-0.035656,3.3739262&plusmn;0.0356561,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,-1.72992,0.034311,-0.034311,-1.7299200&plusmn;0.0343109,0.329641,-0.708536,0.623950,-1895436878
3,4,KIC 8540376 c,c,KOI-7892,NaN,NaN,TIC 123449386,2015-12,2015,Transit,Space,Kepler,Kepler CCD Array,0.95 m Kepler Telescope,<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,282.377540,18h49m30.61s,44.694569,+44d41m40.45s,74.17444,19.02783,293.09772,67.14400,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,75.20,38.90,-1.10,0,"<div><span class=supersubNumber"">75.20</span><...",<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,

In [12]:
min_mag = 8
max_mag = 13.3
max_trandur = 3           # transit duration in hours
max_period = 7            # period in days
min_transit_depth = 0.9  
num_to_show = 20           # number of transits to calculate observation requests for

planet_dict = filter_data(df, min_mag, max_mag, max_trandur, max_period, min_transit_depth, num_to_show)
print("\nTop 5 currently-imageable planets (by Exoplanet Watch rank) are below:\n")
df.head()

3048 rows to start
647 rows with appropriate RA: if abs(dec) < 50deg, (20.6deg < ra < 210.6deg)
418 rows with appropriate RA and mag between 8 and 13.3
182 rows with appropriate RA, mag btwn 8 and 13.3, and transit duration < 3 hours
146 rows with appropriate RA, mag btwn 8 and 13.3, transit duration < 3 hours, and period < 7 days
55 rows with appropriate RA, mag btwn 8 and 13.3, transit duration < 3 hours, period < 7 days, and transit depth > 0.9

Eligible Planets:

HAT-P-12 b	HAT-P-20 b	HAT-P-22 b	HAT-P-3 b	HAT-P-30 b	HAT-P-36 b	
HAT-P-56 b	HATS-24 b	HATS-33 b	HIP 65 A b	K2-121 b	K2-224 b	
K2-228 b	K2-229 b	K2-29 b		KELT-14 b	KELT-23 A b	Qatar-1 b	
TOI-157 b	TOI-564 b	TOI-905 b	WASP-101 b	WASP-104 b	WASP-11 b	
WASP-121 b	WASP-139 b	WASP-140 b	WASP-142 b	WASP-145 A b	WASP-160 B b	
WASP-164 b	WASP-168 b	WASP-175 b	WASP-18 b	WASP-183 b	WASP-23 b	
WASP-25 b	WASP-31 b	WASP-33 b	WASP-34 b	WASP-36 b	WASP-41 b	
WASP-42 b	WASP-43 b	WASP-46 b	WASP-49 b	WASP-50 b	WASP-64 b	
WASP-65 b	WASP-77 A 

,rank,pl_name,pl_letter,hostname,hd_name,hip_name,tic_id,disc_pubdate,disc_year,discoverymethod,disc_locale,disc_facility,disc_instrument,disc_telescope,disc_refname,ra,rastr,dec,decstr,glon,glat,elon,elat,ra_reflink,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbperstr,pl_orbper_reflink,pl_orblpererr1,pl_orblper,pl_orblpererr2,pl_orblperlim,pl_orblperstr,pl_orblper_reflink,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,...,st_densstr,st_dens_reflink,st_rad,st_raderr1,st_raderr2,st_radlim,st_radstr,st_rad_reflink,ttv_flag,ptv_flag,tran_flag,rv_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,pul_flag,sy_snum,sy_pnum,sy_mnum,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,pl_nnotes,sy_pm,sy_pmerr1,sy_pmerr2,sy_pmstr,sy_pm_reflink,sy_pmra,sy_pmraerr1,sy_pmraerr2,sy_pmrastr,x,y,z,htm20
0,1,HIP 41378 e,e,HIP 41378,NaN,HIP 41378,TIC 366443426,2016-08,2016,Transit,Space,K2,Kepler CCD Array,0.95 m Kepler Telescope,<a refstr=VANDERBURG_ET_AL__2016 href=https://...,126.615828,08h26m27.80s,10.080371,+10d04m49.33s,214.52630,25.64577,126.46324,-8.84499,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,131.00,61.00,-36.00,0,"<div><span class=supersubNumber"">131</span><sp...",<a refstr=VANDERBURG_ET_AL__2016 href=https://...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.680&plusmn0.064,<a refstr=BERARDO_ET_AL__2019 href=https://ui....,1.34,0.03,-0.03,0,1.343&plusmn0.032,<a refstr=BERARDO_ET_AL__2019 href=https://ui....,0,0,1,0,0,0,0,0,0,0,1,5,0,1,0,0,0,0,0.0,48.142537,0.104124,-0.104124,48.1425366&plusmn;0.1041239,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,-48.14250,0.104124,-0.104124,-48.142500&plusmn;0.104124,-0.587239,0.790262,0.175029,1036383337
1,2,KIC 5951458 b,b,KIC 5951458,NaN,NaN,TIC 121796770,2015-12,2015,Transit,Space,Kepler,Kepler CCD Array,0.95 m Kepler Telescope,<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,288.991622,19h15m57.99s,41.223068,+41d13m23.05s,72.73419,13.23719,302.08759,62.56270,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,1320.10,12401.80,-152.50,0,"<div><span class=supersubNumber"">1320.10</span...",<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"<div><span class=supersubNumber"">0.2578967</sp...",<a refstr=STASSUN_ET_AL__2019 href=https://ui....,1.52,0.82,-0.82,0,1.52&plusmn0.82,<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,3.0,11.485196,0.041218,-0.041218,11.4851964&plusmn;0.0412180,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,7.38060,0.038297,-0.038297,7.3806000&plusmn;0.0382966,0.244772,-0.711207,0.658992,-1395191073
2,3,KIC 3558849 b,b,KIC 3558849,NaN,NaN,TIC 138888805,2015-12,2015,Transit,Space,Kepler,Kepler CCD Array,0.95 m Kepler Telescope,<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,294.949838,19h39m47.96s,38.605187,+38d36m18.67s,72.43389,7.98057,309.36051,58.68276,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,1322.30,386.10,-11.20,0,"<div><span class=supersubNumber"">1322.3</span>...",<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"<div><span class=supersubNumber"">0.48871</span...",<a refstr=Q1_Q17_DR25_KOI_TABLE href=https://e...,1.01,0.11,-0.11,0,1.01&plusmn0.11,<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1.0,3.373926,0.035656,-0.035656,3.3739262&plusmn;0.0356561,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,-1.72992,0.034311,-0.034311,-1.7299200&plusmn;0.0343109,0.329641,-0.708536,0.623950,-1895436878
3,4,KIC 8540376 c,c,KOI-7892,NaN,NaN,TIC 123449386,2015-12,2015,Transit,Space,Kepler,Kepler CCD Array,0.95 m Kepler Telescope,<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,282.377540,18h49m30.61s,44.694569,+44d41m40.45s,74.17444,19.02783,293.09772,67.14400,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,75.20,38.90,-1.10,0,"<div><span class=supersubNumber"">75.20</span><...",<a refstr=WANG_ET_AL__2015 href=https://ui.ads...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,

In [13]:
# Stuff specific to Fall 2021 Astrophysics students:

stu_dict = {"WASP-175 b": "Jack", "HIP 65 A b": "Ella", "WASP-50 b": "Alex", 
            "WASP-23 b": "Adela", "WASP-121 b": "Elane", "WASP-145 A b": "Trent", 
            "CoRoT-2 b": "Keshav", "WASP-98 b": "Carolina", "WASP-24 b": "Xola", 
            "WASP-52 b": "Aisha", "WASP-5 b": "Harrison", "WASP-95 b": "Joey", 
            "HAT-P-23 b": "Layla" }

not_found = []
for planet in stu_dict:
  student = stu_dict[planet]
  if planet not in planet_dict:
    coords = ras[planet]
    not_found.append(str(f"****** {student}'s planet ({planet}, coords {coords}) not currently imageable\n"))
  else:
    data = planet_dict[planet]
    print(student+"\n"+data)
for unfound in not_found:
  print(unfound)

Jack
Rank 1642 planet WASP-175 b (166.3188, -34.1227)
star mag 13.0
parallax 1.83 Bmag 13.3, Vmag 13.0, B-V 0.3, Spectral type F7
t0 2457143.789 +/- 0.00034
period 3.0653 +/- 1e-06
transit depth 1.064
transit duration 2.68
Request 158 images with exposure time 92 s.
Based on WASP-175 b parameters midpoint 2457143.789 +/- 0.00034 period 3.0653 +/- 1.1e-06:
epoch 799 : midpoint 2459592.95627 (2022-01-13 08:21:44.928 - 2022-01-13 13:32:18.528) UTC (uncertainty 0.02 hours)
epoch 800 : midpoint 2459596.02156 (2022-01-16 09:55:45.984 - 2022-01-16 15:06:19.584) UTC (uncertainty 0.02 hours)
epoch 801 : midpoint 2459599.08685 (2022-01-19 11:29:47.040 - 2022-01-19 16:40:20.640) UTC (uncertainty 0.02 hours)
epoch 802 : midpoint 2459602.15214 (2022-01-22 13:03:48.096 - 2022-01-22 18:14:21.696) UTC (uncertainty 0.02 hours)
epoch 803 : midpoint 2459605.21743 (2022-01-25 14:37:49.152 - 2022-01-25 19:48:22.752) UTC (uncertainty 0.02 hours)
epoch 804 : midpoint 2459608.28272 (2022-01-28 16:11:50.208 - 

In [ ]:
# If the student insists on imaging a planet that is not currently image-able or that
# did not appear in Rob's spreadsheet above.

def hypothetical_image_request(student, planet, mag, plx, bmag, spec, t0, t0_unc, period, 
                               period_unc, ra, dec, transit_depth, transit_duration, 
                               num_to_show):

  exp_time = round(60 * (2.512**(mag - 12.5)),0) # seconds
  t_image = exp_time + 14
  baseline = 2                          # hours
  wiggle = 0.5                          # hours
  n_images = 3600*(transit_duration + baseline)/t_image 
  window = (0.5 * transit_duration) + (0.5 * baseline) + (0.5 * wiggle)  # hours
  window = window/24.0   # days

  out = str(f"{student} Planet {planet} ({round(ra,4)}, {round(dec,4)})\nstar mag {round(mag,1)}\n")
  out = out + str(f"parallax {round(plx,2)} Bmag {round(bmag,1)}, Vmag {round(mag,1)}, B-V {round((bmag-mag),1)}, Spectral type {spectype}\n")
  out = out + str(f"t0 {round(t0,4)} +/- {round(t0_unc,6)}\nperiod {round(period,4)} +/- {round(period_unc,6)}\n")
  out = out + str(f"transit depth {round(transit_depth,3)}\ntransit duration {round(transit_duration,2)}\n")
  out = out + str(f"Request {int(n_images)} images with exposure time {int(exp_time)} s.\n")
  out = out + str(f"Window {round(window*24,2)}\n")
  out = out + str(f"RA {ra} Dec {dec}\n")
  out = out + upcoming_transits(planet, t0, t0_unc, period, period_unc, ra, dec, num_to_show, window)

  return(out)

###################################

num_to_show = 5

out_strs = []

student, planet, mag, plx, bmag, spectype = "Xola", "XO-1 b", 11.251, 6.085, 11.878, "G1 V"
t0, t0_unc, period, period_unc = 2453887.74774, 0.00022, 3.94150685, 0.00000018
ra, dec, transit_depth, transit_duration = 240.5493596272596, 28.1695610117719, 1.758, 2.992

out_strs.append(hypothetical_image_request(student, planet, mag, plx, bmag, spectype, t0, t0_unc, period, period_unc, ra, dec, transit_depth, transit_duration, num_to_show))

student, planet, mag, plx, bmag, spectype = "Jack", "WASP-175 b", 12.959, 1.826, 13.288, "F7"
t0, t0_unc, period, period_unc = 2457143.78886, 0.00034, 3.0652907, 0.0000016
ra, dec, transit_depth, transit_duration = 166.3189259742813, -34.1227712539256, 1.064, 2.676

out_strs.append(hypothetical_image_request(student, planet, mag, plx, bmag, spectype, t0, t0_unc, period, period_unc, ra, dec, transit_depth, transit_duration, num_to_show))

student, planet, mag, plx, bmag, spectype = "Keshav", "WASP-12 b", 11.569, 2.31224, 12.139, "unknown"
t0, t0_unc, period, period_unc = 2456176.66825800, 0.00007765, 1.09142, 0.00000014432
ra, dec, transit_depth, transit_duration = 97.6366528287125, 29.6722962039328, 1.389, 2.9959

out_strs.append(hypothetical_image_request(student, planet, mag, plx, bmag, spectype, t0, t0_unc, period, period_unc, ra, dec, transit_depth, transit_duration, num_to_show))

for out_str in out_strs:
  print(out_str)




Xola Planet XO-1 b (240.5494, 28.1696)
star mag 11.3
parallax 6.08 Bmag 11.9, Vmag 11.3, B-V 0.6, Spectral type G1 V
t0 2453887.7477 +/- 0.00022
period 3.9415 +/- 0.0
transit depth 1.758
transit duration 2.99
Request 544 images with exposure time 19 s.
Window 2.75
RA 240.5493596272596 Dec 28.1695610117719
Based on XO-1 b parameters midpoint 2453887.7477 +/- 0.00022 period 3.9415 +/- 1.8e-07:
epoch 1427 : midpoint 2459512.27801 (2021-10-24 15:55:34.464 - 2021-10-24 21:25:05.664) UTC (uncertainty 0.01 hours)
epoch 1428 : midpoint 2459516.21952 (2021-10-28 14:31:20.928 - 2021-10-28 20:00:52.128) UTC (uncertainty 0.01 hours)
epoch 1429 : midpoint 2459520.16103 (2021-11-01 13:07:07.392 - 2021-11-01 18:36:38.592) UTC (uncertainty 0.01 hours)
epoch 1430 : midpoint 2459524.10254 (2021-11-05 11:42:53.856 - 2021-11-05 17:12:25.056) UTC (uncertainty 0.01 hours)
epoch 1431 : midpoint 2459528.04404 (2021-11-09 10:18:39.456 - 2021-11-09 15:48:10.656) UTC (uncertainty 0.01 hours)


Jack Planet WASP-1